<a href="https://colab.research.google.com/github/ozturkcemal/SupplyChainAnalytics/blob/main/03_FacilityLocation_CenterOfGravity/ManualCenterSelection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# Import necessary libraries
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

## Haversine Formula

The Haversine formula is used to calculate the distance between two points on the Earth's surface given their latitude and longitude.

$
d = 2r \times \arcsin\left(\sqrt{\sin^2\left(\frac{\Delta\phi}{2}\right) + \cos(\phi_1) \cdot \cos(\phi_2) \cdot \sin^2\left(\frac{\Delta\lambda}{2}\right)}\right)
$

Where:

- $\phi_1$ and $\phi_2$ are the latitudes of the two points in radians.
- $\lambda_1$ and $\lambda_2$ are the longitudes of the two points in radians.
- $\Delta\phi = \phi_2 - \phi_1$ is the difference in latitudes.
- $\Delta\lambda = \lambda_2 - \lambda_1$ is the difference in longitudes.
- $r$ is the Earth’s radius (mean radius = 6,371 kilometers).

In [6]:

# Haversine function to compute the distance in kilometers
def haversine(lat1, lon1, lat2, lon2):
    # Convert degrees to radians
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])

    # Haversine formula
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat / 2) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2) ** 2
    c = 2 * np.arcsin(np.sqrt(a))

    # Radius of Earth in kilometers
    r = 6371  # kilometers
    return c * r

In [7]:
# Data from the table
data = {
    'City #': list(range(1, 26)),  # Create City # from 1 to 25
    'Latitude': [-16.6, -16.7, -16.8, -17, -17, -17.2, -17.5, -17.4, -17.5, -17.8, -17.9, -18, -18.4, -18.9, -19.3,
                 -19.4, -19.4, -19.9, -20.3, -21.2, -21.6, -22.6, -23.4, -24, -24.9],
    'Longitude': [154.8, 156.8, 153.2, 154, 152, 144.9, 155.7, 147.1, 141.1, 155.1, 153.8, 144.6, 142.4, 156.8, 148.3,
                  152.9, 142.8, 143.7, 152.5, 143.7, 155.6, 140.1, 155.8, 144.4, 146.4],
    'Population': [1655000, 2300000, 601000, 1385000, 1230000, 665000, 664000, 885000, 1116000, 636000, 1200000, 148000,
                   854000, 1473000, 615000, 1145000, 627000, 542000, 379000, 964000, 546000, 706000, 727000, 669000, 931000],
    'Distance_to_Capital': [0] * 25  # Assign default value of 0 for Distance_to_Capital
}

# Create a pandas DataFrame
df = pd.DataFrame(data)

In [8]:
# Calculate the averages for Capital Location
Capital_Latitude = df['Latitude'].mean()
Capital_Longitude = df['Longitude'].mean()

# Compute the distance to the capital for each city using the Haversine formula
df['Distance_to_Capital'] = df.apply(
    lambda row: haversine(row['Latitude'], row['Longitude'], Capital_Latitude, Capital_Longitude), axis=1)

# Compute the Weighted_Average_Distance
weighted_distance_sum = (df['Population'] * df['Distance_to_Capital']).sum()  # Sum of (Population * distance)
total_population = df['Population'].sum()  # Total population
Weighted_Average_Distance = weighted_distance_sum / total_population

# Calculate the percentage of cities within 500 km
cities_within_500km = df[df['Distance_to_Capital'] <= 500].shape[0]
total_cities = df.shape[0]
percent_cities_within_500km = (cities_within_500km / total_cities) * 100

# Output the results
print(f"Weighted Average Distance to Capital: {Weighted_Average_Distance:.2f} km")
print(f"Percentage of Cities within 500 km of the Capital: {percent_cities_within_500km:.2f}%")

Weighted Average Distance to Capital: 625.36 km
Percentage of Cities within 500 km of the Capital: 28.00%


In [ ]:
# Initialize the plot
plt.figure(figsize=(10, 6))

# Create the scatter plot using seaborn for cities
sns.scatterplot(x='Latitude', y='Longitude', size='Population', data=df, sizes=(20, 1000), legend=False)

# Plot the capital location with a distinct marker and label it
plt.scatter(Capital_Latitude, Capital_Longitude, color='red', s=300, marker='X', label='Capital')

# Add labels to each point for City #
for i in range(df.shape[0]):
    plt.text(df['Latitude'][i], df['Longitude'][i], f"City {df['City #'][i]}", fontsize=9, ha='right')

# Add labels and title
plt.title("Scatter Plot of Latitude vs Longitude with City # Labels and Capital Location", fontsize=14)
plt.xlabel("Latitude", fontsize=12)
plt.ylabel("Longitude", fontsize=12)

# Add legend, forcing the label to be displayed
plt.legend(loc='upper right')

# Show the plot
plt.show()

# Print the DataFrame to see the updated Distance_to_Capital column
print(df)

In [ ]:
# Export the DataFrame to a CSV file
output_file = 'city_data_with_distances.csv'
df.to_csv(output_file, index=False)

print(f"DataFrame exported to {output_file}")